In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from palettable.tableau import Tableau_10
from cycler import cycler

In [3]:
scale = 3
almost_black = '#262626'
plt.rc('lines', linestyle=None, markeredgewidth=0.15)
plt.rc('axes', prop_cycle=cycler(color=Tableau_10.mpl_colors),
       labelsize=9*scale, edgecolor=almost_black)
markers = ['o', 's', 'd', '^', 'x', '+', 'v', '<', '>', '*']
assert len(markers) == len(Tableau_10.mpl_colors)
def mfunc(syms):
    while True:
        for s in syms:
            yield s
plt.rc('xtick', labelsize=9*scale)
plt.rc('ytick', labelsize=9*scale)
plt.rc('legend', fontsize=8*scale)
plt.rc('font', family='serif', serif="Times New Roman", size=9*scale)
plt.rc('text', color=almost_black)
plt.rc('mathtext', fontset='stix')
plt.rc('xtick.major', size=10, width=1)
plt.rc('xtick.minor', size=5, width=1)
plt.rc('ytick.major', size=10, width=1)
plt.rc('ytick.minor', size=5, width=1)
plt.rc('figure', dpi=300, autolayout=True)

In [4]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('methyl-valerate-78135e2c2c7c.json',
                                                               scope)
gc = gspread.authorize(credentials)
ws = gc.open_by_key('1UFdGzEyx7PsKprPybi-lkbuWYS8BAS-cBm2IKSKY-UA').sheet1
recs = ws.get_all_records(empty2zero=True)
data = pd.DataFrame(recs)

phi_100_15_bar = data[(data['Equivalence Ratio'] == 1.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1E-2))][::-1]
phi_100_30_bar = data[(data['Equivalence Ratio'] == 1.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1E-2))][::-1]

In [10]:
hadj_10_bar = pd.read_csv('ignition-delay-comparison/Hadj-Ali-10-bar-MHEX.csv', skipinitialspace=True)
hadj_13_bar = pd.read_csv('ignition-delay-comparison/Hadj-Ali-13-bar-MHEX.csv', skipinitialspace=True)
kumar_MB = pd.read_csv('ignition-delay-comparison/Kumar-MB.csv', skipinitialspace=True)

In [12]:
kumar_MB['1000/Tc (1/K)'] = 1000.0/kumar_MB['Compressed Temperature (K)']
hadj_10_bar['1000/Tc (1/K)'] = 1000.0/hadj_10_bar['Compressed Temperature (K)']
hadj_13_bar['1000/Tc (1/K)'] = 1000.0/hadj_13_bar['Compressed Temperature (K)']

In [15]:
kumar_15_bar = kumar_MB[(kumar_MB['Equivalence Ratio'] == 1.00) &
                        (kumar_MB['Compressed Pressure (bar)'] == 15)][::-1]
kumar_30_bar = kumar_MB[(kumar_MB['Equivalence Ratio'] == 1.00) &
                        (kumar_MB['Compressed Pressure (bar)'] == 30)][::-1]

In [26]:
fig, ax = plt.subplots(figsize=[14, 9])
ax.set_yscale('log')

plot_opts = {'markersize': 15}

mark = mfunc(markers)
ax.plot(kumar_15_bar['1000/Tc (1/K)'], kumar_15_bar['Ignition Delay (ms)'], marker=next(mark), label='MB [10]', **plot_opts)
ax.plot(phi_100_15_bar['1000/Tc (1/K)'], phi_100_15_bar['Ignition Delay (ms)'], marker=next(mark), label='MV (this work)', **plot_opts)
ax.plot(hadj_13_bar['1000/Tc (1/K)'], hadj_13_bar['Ignition Delay (ms)'], marker=next(mark), label='MHEX [9]', **plot_opts)

ax.set_xlabel('$1000/T_C$, 1/K')
ax.set_ylabel('Ignition Delay, ms')
ax.set_ylim(1, 200)
ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.legend(loc='best', handletextpad=0.0, numpoints=1)

# Create the temperature axes on the top
def convert_inv_temp(temps):
    """Convert a list of temperatures to inverse temperature"""
    return [1000.0/temp for temp in temps]

ax_temp = ax.twiny()

# Set the major tick marks in the temperature scale and convert to inverse scale
major_temps = np.arange(1100, 600, -100)
major_ticks = convert_inv_temp(major_temps)

# Set the interval for the minor ticks and compute the minor ticks
minor_interval = 20
minor_ticks = []
for maj in major_temps:
    minor_ticks.extend(convert_inv_temp([maj - i*minor_interval for i in range(5)]))
    
# Set the ticks on the axis. Note that the limit setting must be present and must be after setting the ticks
# so that the scale is correct
ax_temp.set_xticks(major_ticks)
ax_temp.set_xticks(minor_ticks, minor=True)
ax_temp.set_xticklabels(['{:d} K'.format(temp) for temp in major_temps])
ax_temp.set_xlim(ax.get_xlim())

(1.0, 1.6000000000000001)

In [27]:
fig.savefig('ignition-delay-comparison.png')